# Fonctions pour generer des csv enedis avec DPE par departements

In [3]:
# import libraries
import pandas as pd
import numpy as np
import os
# chemin du dataset national
DATA_PATH = 'drive/MyDrive/enedis-wagon/data/'
DATA_CSV = 'consommation-annuelle-residentielle-par-adresse.csv'
# import du csv pour l'exploration
df = pd.read_csv(os.path.join(DATA_PATH, DATA_CSV), sep=';', low_memory=False)

In [4]:
import requests
from io import BytesIO

# fonction pour filtrer le dataset en fonction du code departement
def filter_df_by_code_departement(code_departement):
    df.update(df[df['code_departement'] == code_departement])
    df.to_csv(os.path.join(DATA_PATH, f'enedis_{code_departement}.csv'), index=False)

    # fonction pour l'utilisation de l'api du gouv pour identifier identifiant ban par rapport a l'adresse, nom et code commune
    input_filepath = os.path.join(DATA_PATH, f'enedis_{code_departement}.csv')
    url_csv = 'https://api-adresse.data.gouv.fr/search/csv/'
    data = {'result_columns': ['result_id', 'result_score'],
            'columns': ['adresse', 'nom_commune'],
            'citycode': 'code_commune'}

    def get_ban_id_from_csv(input_filepath,
                            output_filepath="temp.csv",
                            url=url_csv,
                            data=data):

        with open(input_filepath, "rb") as csv_file:
            files = {'data': csv_file}
            r = requests.post(url, files=files, data=data)
            pd.read_csv(BytesIO(r.content)).to_csv(output_filepath, index=False)
    get_ban_id_from_csv(input_filepath)

    # importation du fichier temp.csv pour filter sur la fiabilitée de la correspondance
    df_clean_ban_formated = pd.read_csv('temp.csv')
    threshold = 0.75
    df_clean_ban_formated = df_clean_ban_formated[df_clean_ban_formated['result_score'] >= threshold]

    # creation du csv
    df_clean_ban_formated.to_csv(os.path.join(DATA_PATH, f'enedis_{code_departement}.csv'), index=False)

    # nettoyage du fichier temp
    file_path = 'temp.csv'
    if os.path.exists(file_path):
        os.remove(file_path)
    else:
        print(f"Le fichier {file_path} n'existe pas.")

    # Compte rendu
    print(f"\nConcordance avec {threshold} de score:\n{df_clean_ban_formated['result_id'].count()}")
    print(f"\nAdresses perdues:\n{57678 - df_clean_ban_formated['result_id'].count()} / 57678")
    print(f"\nNombre d'adresse unique:\n{df_clean_ban_formated['result_id'].nunique()}\n")
    # renommage des columns
    df_clean_ban_formated.rename(columns={'result_id': 'Identifiant__BAN'}, inplace=True)
    # garder les columns utile
    df_clean_ban_formated = df_clean_ban_formated[['annee', 'nombre_de_logements', 'consommation_annuelle_totale_de_l_adresse_mwh', 'consommation_annuelle_moyenne_par_site_de_l_adresse_mwh', 'consommation_annuelle_moyenne_de_la_commune_mwh', 'Identifiant__BAN']]
    df_clean_ban_formated.reset_index(drop=True, inplace=True)
    # export enedis 13 final
    df_clean_ban_formated.to_csv(os.path.join(DATA_PATH, f'enedis_{code_departement}.csv'), index=False)

    return f'enedis_{code_departement}.csv créé'

In [5]:
# Filter the DataFrame based on the 'code_departement' column and update the DataFrame in place
filter_df_by_code_departement('59')

SSLError: HTTPSConnectionPool(host='api-adresse.data.gouv.fr', port=443): Max retries exceeded with url: /search/csv/ (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2427)')))

In [ ]:
pd.read_csv(os.path.join(DATA_PATH, 'enedis_59.csv'))